In [ ]:
import os
from gpuutils import GpuUtils
GpuUtils.allocate(gpu_count=1, framework='keras')

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True) 

import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from sklearn.preprocessing import MinMaxScaler

#tf.random.set_seed(
#    42
#)


In [ ]:
df_test = pd.read_csv('/mnt/md0/aholmberg/data/raytrace_samples_random.csv')
df = pd.read_csv('/mnt/md0/aholmberg/data/raytrace_samples_sobol_21.csv')
#df['launch_angle'] = np.arctan2(df['launch_vec_r'].to_numpy(), df['launch_vec_z'].to_numpy())
#df['recieve_angle'] = np.arctan2(df['recieve_vec_r'].to_numpy(), df['recieve_vec_z'].to_numpy())
#df.to_csv('/mnt/md0/aholmberg/data/raytrace_samples_angle.csv', index=False)

In [ ]:
sc_pos_r = df['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df['antenna_pos_z'].to_numpy().astype(np.float32)
x = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

sc_pos_r = df_test['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df_test['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df_test['antenna_pos_z'].to_numpy().astype(np.float32)
x_test = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

travel_time = df['travel_time'].to_numpy().astype(np.float32)
path_length = df['path_length'].to_numpy().astype(np.float32)
launch = df['launch_angle'].to_numpy().astype(np.float32)
recieve = df['recieve_angle'].to_numpy().astype(np.float32)
y = np.stack((travel_time, path_length, launch, recieve), axis=1)

travel_time = df_test['travel_time'].to_numpy().astype(np.float32)
path_length = df_test['path_length'].to_numpy().astype(np.float32)
launch = df_test['launch_angle'].to_numpy().astype(np.float32)
recieve = df_test['recieve_angle'].to_numpy().astype(np.float32)
y_test = np.stack((travel_time, path_length, launch, recieve), axis=1)


unique, index, count = np.unique(x, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))
x[index[count == 1], :]
x_new = np.delete(x, index[count == 1], axis=0)
y_new = np.delete(y, index[count == 1], axis=0)
unique, index, count = np.unique(x_new, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))

unique, index, count = np.unique(x_test, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))
x_test[index[count == 1], :]
x_new_test = np.delete(x_test, index[count == 1], axis=0)
y_new_test = np.delete(y_test, index[count == 1], axis=0)
unique, index, count = np.unique(x_new_test, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))


In [ ]:
#x_new  = x_new_1[0::2,:]
#x_train = x_new[:int(x_new.shape[0]*0.9)]
#x_test = x_new[int(x_new.shape[0]*0.9):]
x_train = x_new[0::2,:]
x_test = x_new_test[0::2,:]

#y_temp1  = y_new_1[0::2,:]
#y_temp2  = y_new_1[1::2,:]
#y_new = np.zeros((y_temp1.shape[0], 8))

y_temp1  = y_new[0::2,:]
y_temp2  = y_new[1::2,:]
y_temp1_test  = y_new_test[0::2,:]
y_temp2_test  = y_new_test[1::2,:]
y_new = np.zeros((y_temp1.shape[0], 8))
y_new_test = np.zeros((y_temp1_test.shape[0], 8))


for i in range(4):
    y_new[:,2*i] = y_temp1[:,i]
    y_new[:,2*i+1] = y_temp2[:,i]
    y_new_test[:,2*i] = y_temp1_test[:,i]
    y_new_test[:,2*i+1] = y_temp2_test[:,i]

y_train = y_new
y_test = y_new_test
#y_train = y_new[:int(y_new.shape[0]*0.9)]
#y_test = y_new[int(y_new.shape[0]*0.9):]

scaler_x = MinMaxScaler(feature_range=(0,1))
#scaler_x = StandardScaler()
scaler_x.fit(x_train)
norm_x_train = scaler_x.transform(x_train)
norm_x_test = scaler_x.transform(x_test)

scaler_y = MinMaxScaler(feature_range=(0,1))
#scaler_y = StandardScaler()
scaler_y.fit(y_train)
norm_y_train = scaler_y.transform(y_train)
norm_y_test = scaler_y.transform(y_test)



In [ ]:
"""
temp1 = typ[0::2]
temp2 = typ[1::2]
print(temp1, temp2, typ)
print(temp1.shape, temp2.shape, typ.shape)
unique, counts = np.unique(temp1, return_counts=True)
a = dict(zip(unique, counts))
unique, counts = np.unique(temp2, return_counts=True)
b = dict(zip(unique, counts))
print(a, b)
unique, counts = np.unique(features_train[:,0], return_counts=True)
#c = dict(zip(unique, counts))
print(max(counts))
"""
pass

In [ ]:
#model = keras.Model(inputs=inputs, outputs=[output_time, output_length, output_launch, output_recieve], name="full_model")
from raytrace_model_def import get_simple_model, get_multiple_output_model, get_no_skip_model, get_skip_model
activation = 'relu'
#depth = 4
width = 64
opt = keras.optimizers.Adam(learning_rate=1e-3)
#model = get_simple_model(width, depth, activation, optimizer=opt)

model = get_no_skip_model(width, activation=activation, loss='mae')

#model = get_multiple_output_model(width, depth, activation=activation)

In [ ]:

print(model.summary())

In [ ]:
early_stoping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5
)

def scheduler(epoch, lr):
    if epoch <= 2:
        return 1e-2
    elif epoch <= 8:
        return 1e-3
    else:
       return lr * tf.math.exp(-0.1)

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
#model.fit(norm_features_train, [norm_time_train, norm_length_train, norm_launch_train, norm_recieve_train], epochs=5)
model.fit(
    norm_x_train[:,:],
    norm_y_train[:,:],#[norm_y_train[:,:2], norm_y_train[:,2:4], norm_y_train[:,4:]],
    epochs=20,
    batch_size=128,
    validation_split=0.1,  
    callbacks=[lr_scheduler]
    )

In [ ]:
name = 'multi-out-bigger-skip-weight-v2-2'
model_path = '/mnt/md0/aholmberg/models/' + name
model = keras.models.load_model(model_path)
print(model.summary())
scaler_path_x = model_path + "-x-scaler.pkl"
scaler_path_y = model_path + "-y-scaler.pkl"
from pickle import load
scaler_x = load(open(scaler_path_x, 'rb'))
scaler_y = load(open(scaler_path_y, 'rb'))

In [ ]:
sc_pos_r = df_test['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df_test['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df_test['antenna_pos_z'].to_numpy().astype(np.float32)
x_test = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

travel_time = df_test['travel_time'].to_numpy().astype(np.float32)
path_length = df_test['path_length'].to_numpy().astype(np.float32)
launch = df_test['launch_angle'].to_numpy().astype(np.float32)
recieve = df_test['recieve_angle'].to_numpy().astype(np.float32)
y_test = np.stack((travel_time, path_length, launch, recieve), axis=1)


unique, index, count = np.unique(x_test, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))
x_test[index[count == 1], :]
x_new_test = np.delete(x_test, index[count == 1], axis=0)
y_new_test = np.delete(y_test, index[count == 1], axis=0)
unique, index, count = np.unique(x_new_test, return_counts=True, return_index=True, axis=0)
print(unique, index, count)
print(np.unique(count, return_counts=True))
x_test = x_new_test[0::2,:]

y_temp1_test  = y_new_test[0::2,:]
y_temp2_test  = y_new_test[1::2,:]
y_new_test = np.zeros((y_temp1_test.shape[0], 8))


for i in range(4):
    y_new_test[:,2*i] = y_temp1_test[:,i]
    y_new_test[:,2*i+1] = y_temp2_test[:,i]


y_test = y_new_test

norm_x_test = scaler_x.transform(x_test)
norm_y_test = scaler_y.transform(y_test)


In [ ]:
"""
y_test = model(norm_features_test)
y_time = y_test[0].numpy()
y_length = y_test[1].numpy()
y_launch = y_test[2].numpy()
y_recieve = y_test[3].numpy()
"""
temp1 = model(norm_x_test[:int(norm_x_test.shape[0]/2)])
temp2 = model(norm_x_test[int(norm_x_test.shape[0]/2):])
print(temp1[0].shape ,temp1[1].shape, temp1[2].shape,temp2[0].shape ,temp2[1].shape, temp2[2].shape)
y_test_pred = np.zeros((norm_y_test.shape))
#y_test_pred[:int(norm_x_test.shape[0]/2),:] = np.array(model(norm_x_test[:int(norm_x_test.shape[0]/2)]))#.numpy().flatten()
#y_test_pred[int(norm_x_test.shape[0]/2):,:] = np.array(model(norm_x_test[int(norm_x_test.shape[0]/2):]))#.numpy().flatten()

y_test_pred[:int(norm_x_test.shape[0]/2),:] = np.concatenate((temp1[0], temp1[1], temp1[2], temp1[3]), axis=1)
y_test_pred[int(norm_x_test.shape[0]/2):,:] = np.concatenate((temp2[0], temp2[1], temp2[2], temp2[3]), axis=1)
y_test_inv = scaler_y.inverse_transform(y_test_pred)

print(y_test_pred.shape)
y_time = y_test_pred[:,:2]
y_length = y_test_pred[:,2:4]
y_launch = y_test_pred[:,4:6]
y_recieve = y_test_pred[:,6:8]


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from radiotools import plthelpers as php


In [ ]:
plt.rcParams.update({'font.size': 14})


for sol in range (8):
    data = pd.DataFrame({'x-val':y_test[:,sol] , 'y-val':y_test_inv[:,sol]- y_test[:,sol]})
    g = sns.JointGrid(data=data,
                    x="x-val",
                    y="y-val")
    
    g.plot_joint(sns.scatterplot, s=100, alpha=.5)
    g.plot_marginals(sns.histplot, kde=True)
    if sol == 0 or sol == 1:
        g.set_axis_labels(xlabel='True traveltime [ns]', ylabel='Error travel time [ns] ')
    elif sol == 2 or sol == 3:
        g.set_axis_labels(xlabel='True path length [m]', ylabel='Error path length [m] ')
    elif sol == 4 or sol == 5:
        g.set_axis_labels(xlabel='True launch angle [$^\circ$]', ylabel='Error launch angle [$^\circ$] ')
    elif sol == 6 or sol == 7:
        g.set_axis_labels(xlabel='True recieve angle [$^\circ$]', ylabel='Error recieve angle [$^\circ$] ')
    g.savefig(f'/mnt/md0/aholmberg/plots/raytrace/jointgrid/sol_{sol}.png')

In [ ]:
plt.scatter(norm_y_test[:,0], y_time[:,0]- norm_y_test[:,0])
plt.xlabel('Analytically calculated travel time (ns)', size=14)
plt.ylabel('Network prediction error (ns)', size=14)
#plt.savefig('model_pred_1.svg')

In [ ]:
plt.scatter(norm_y_test[:,0], y_test_pred[:,0])
plt.xlabel('Analytically calculated travel time (ns)', size=14)
plt.ylabel('Network prediction (ns)', size=14)
#plt.savefig('model_pred_2.svg')

In [ ]:
diff_deg = y_test - y_test_inv

#diff = y_test - y_test_inv
#diff_deg = np.copy(diff)
#diff_deg[:,4:] = np.degrees(diff_deg[:, 4:])

sol = ['time_sol_1:', 
       'time_sol_2:',
       'length_sol_1:',
       'length_sol_2:',
       'launch_sol_1:',
       'launch_sol_2:',
       'recieve_sol_1:',
       'recieve_sol_2:']

for i in range(8):
    #mean = np.mean(diff[:,i])
    #std = np.std(diff[:,i])
    mean = np.mean(diff_deg[:,i])
    med = np.median(diff_deg[:,i])
    std = np.std(diff_deg[:,i])
    print(sol[i] + f' mean: {mean:.4f} median: {med:.4f}  std: {std:.4f}')

In [ ]:
x_lab = ['travel time [ns]', 'path length [m]', 'launch angle $[^\circ]$', 'recieve angle $[^\circ]$']
for i in range(8):
    std = np.std(diff_deg[:,i])
    bins = np.linspace(-3*std, 3*std, 100)
    fix, ax = php.get_histogram(diff_deg[:,i], bins=bins)
    x_label = 'Error in ' + x_lab[int(i/2)]
    plt.xlabel(x_label)
    plt.savefig(f'/mnt/md0/aholmberg/plots/raytrace/jointgrid/hist_sol_{i}.png')

In [ ]:
np.percentile(diff_deg[:,0], [16, 84])

In [ ]:
for i in range(3):
    print('random: ', np.max(x_test[:,i]), 'sobol: ', np.max(x[:,i]))
    print('random: ', np.min(x_test[:,i]), 'sobol: ', np.min(x[:,i]))